# Fake News Classifier Using LSTM

### Dataset: https://www.kaggle.com/c/fake-news/data#

## Import all Libraries

In [1]:
import pandas as pd
import numpy as np
import nltk
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer ##stemming purpose
import tensorflow as tf
print(tf.__version__)
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix,accuracy_score, classification_report

2.12.0


## Import Datasets & Visualizing

In [2]:
df=pd.read_csv('FakeNewsData/train.csv')

In [3]:
print(df.head(),'\n',df.shape)

   id                                              title              author  \
0   0  House Dem Aide: We Didn’t Even See Comey’s Let...       Darrell Lucus   
1   1  FLYNN: Hillary Clinton, Big Woman on Campus - ...     Daniel J. Flynn   
2   2                  Why the Truth Might Get You Fired  Consortiumnews.com   
3   3  15 Civilians Killed In Single US Airstrike Hav...     Jessica Purkiss   
4   4  Iranian woman jailed for fictional unpublished...      Howard Portnoy   

                                                text  label  
0  House Dem Aide: We Didn’t Even See Comey’s Let...      1  
1  Ever get the feeling your life circles the rou...      0  
2  Why the Truth Might Get You Fired October 29, ...      1  
3  Videos 15 Civilians Killed In Single US Airstr...      1  
4  Print \nAn Iranian woman has been sentenced to...      1   
 (20800, 5)


In [4]:
df.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [5]:
df=df.dropna()

In [6]:
print(df.head(),'\n',df.shape)

   id                                              title              author  \
0   0  House Dem Aide: We Didn’t Even See Comey’s Let...       Darrell Lucus   
1   1  FLYNN: Hillary Clinton, Big Woman on Campus - ...     Daniel J. Flynn   
2   2                  Why the Truth Might Get You Fired  Consortiumnews.com   
3   3  15 Civilians Killed In Single US Airstrike Hav...     Jessica Purkiss   
4   4  Iranian woman jailed for fictional unpublished...      Howard Portnoy   

                                                text  label  
0  House Dem Aide: We Didn’t Even See Comey’s Let...      1  
1  Ever get the feeling your life circles the rou...      0  
2  Why the Truth Might Get You Fired October 29, ...      1  
3  Videos 15 Civilians Killed In Single US Airstr...      1  
4  Print \nAn Iranian woman has been sentenced to...      1   
 (18285, 5)


## Get the Independent & Dependent Features

In [7]:
X=df.drop('label',axis=1)
y=df['label']

In [8]:
print(X.shape,'\n',y.shape)

(18285, 4) 
 (18285,)


In [9]:
## Check whether dataset is balanced or not
y.value_counts()

label
0    10361
1     7924
Name: count, dtype: int64

In [10]:
### Vocabulary size
voc_size=5000

## Dataset Preprocessing

In [11]:
messages=X.copy(deep=True)

In [12]:
print(messages.head(10),'\n',messages['title'][1])

    id                                              title  \
0    0  House Dem Aide: We Didn’t Even See Comey’s Let...   
1    1  FLYNN: Hillary Clinton, Big Woman on Campus - ...   
2    2                  Why the Truth Might Get You Fired   
3    3  15 Civilians Killed In Single US Airstrike Hav...   
4    4  Iranian woman jailed for fictional unpublished...   
5    5  Jackie Mason: Hollywood Would Love Trump if He...   
7    7  Benoît Hamon Wins French Socialist Party’s Pre...   
9    9  A Back-Channel Plan for Ukraine and Russia, Co...   
10  10  Obama’s Organizing for Action Partners with So...   
11  11  BBC Comedy Sketch "Real Housewives of ISIS" Ca...   

                          author  \
0                  Darrell Lucus   
1                Daniel J. Flynn   
2             Consortiumnews.com   
3                Jessica Purkiss   
4                 Howard Portnoy   
5                Daniel Nussbaum   
7                Alissa J. Rubin   
9   Megan Twohey and Scott Shane   
10  

In [13]:
messages.reset_index(inplace=True)

In [14]:
print(messages.head(10),'\n',messages['title'][1])

   index  id                                              title  \
0      0   0  House Dem Aide: We Didn’t Even See Comey’s Let...   
1      1   1  FLYNN: Hillary Clinton, Big Woman on Campus - ...   
2      2   2                  Why the Truth Might Get You Fired   
3      3   3  15 Civilians Killed In Single US Airstrike Hav...   
4      4   4  Iranian woman jailed for fictional unpublished...   
5      5   5  Jackie Mason: Hollywood Would Love Trump if He...   
6      7   7  Benoît Hamon Wins French Socialist Party’s Pre...   
7      9   9  A Back-Channel Plan for Ukraine and Russia, Co...   
8     10  10  Obama’s Organizing for Action Partners with So...   
9     11  11  BBC Comedy Sketch "Real Housewives of ISIS" Ca...   

                         author  \
0                 Darrell Lucus   
1               Daniel J. Flynn   
2            Consortiumnews.com   
3               Jessica Purkiss   
4                Howard Portnoy   
5               Daniel Nussbaum   
6               A

In [15]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [16]:
ps = PorterStemmer()
corpus = []
for i in range(0, len(messages)):
    review = re.sub('[^a-zA-Z]', ' ', messages['title'][i])
    review = review.lower()
    review = review.split()

    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

In [17]:
corpus

['hous dem aid even see comey letter jason chaffetz tweet',
 'flynn hillari clinton big woman campu breitbart',
 'truth might get fire',
 'civilian kill singl us airstrik identifi',
 'iranian woman jail fiction unpublish stori woman stone death adulteri',
 'jacki mason hollywood would love trump bomb north korea lack tran bathroom exclus video breitbart',
 'beno hamon win french socialist parti presidenti nomin new york time',
 'back channel plan ukrain russia courtesi trump associ new york time',
 'obama organ action partner soro link indivis disrupt trump agenda',
 'bbc comedi sketch real housew isi caus outrag',
 'russian research discov secret nazi militari base treasur hunter arctic photo',
 'us offici see link trump russia',
 'ye paid govern troll social media blog forum websit',
 'major leagu soccer argentin find home success new york time',
 'well fargo chief abruptli step new york time',
 'anonym donor pay million releas everyon arrest dakota access pipelin',
 'fbi close hilla

## Onehot Representation

In [18]:
onehot_repr=[one_hot(words,voc_size)for words in corpus]
onehot_repr

[[1235, 827, 2514, 2524, 820, 2474, 4636, 4512, 929, 4746],
 [3668, 3330, 288, 4902, 3549, 4747, 2534],
 [2305, 2980, 4398, 4726],
 [3007, 2729, 4088, 426, 3770, 124],
 [4583, 3549, 1461, 928, 3223, 2613, 3549, 2842, 4598, 2530],
 [1785,
  601,
  4671,
  1983,
  1000,
  4049,
  3166,
  4625,
  4663,
  2828,
  1405,
  2726,
  881,
  1773,
  2534],
 [2458, 1708, 349, 2101, 3959, 4336, 954, 510, 1414, 4605, 3222],
 [1536, 191, 3279, 3590, 1791, 684, 4049, 2453, 1414, 4605, 3222],
 [495, 4518, 288, 2052, 1751, 1886, 3294, 2990, 4049, 2153],
 [2014, 3038, 4757, 4844, 3961, 2317, 4059, 4828],
 [512, 3370, 803, 3557, 2457, 2004, 2718, 2094, 510, 4364, 338],
 [426, 2798, 820, 1886, 4049, 1791],
 [3195, 879, 594, 2020, 4979, 120, 2752, 1631, 4840],
 [301, 302, 2353, 1221, 1437, 2597, 4830, 1414, 4605, 3222],
 [4236, 4451, 674, 1918, 564, 1414, 4605, 3222],
 [1482, 3979, 207, 2095, 3633, 3670, 3184, 4384, 4973, 635],
 [3882, 67, 3330],
 [2677, 17, 2989, 1168, 4049, 3852, 2553, 2534],
 [2695, 321

In [19]:
print(corpus[1],'\n',onehot_repr[1])

flynn hillari clinton big woman campu breitbart 
 [3668, 3330, 288, 4902, 3549, 4747, 2534]


## Embedding Representation

In [20]:
sent_length=20
embedded_docs=pad_sequences(onehot_repr,padding='post',maxlen=sent_length)
print(embedded_docs)

[[1235  827 2514 ...    0    0    0]
 [3668 3330  288 ...    0    0    0]
 [2305 2980 4398 ...    0    0    0]
 ...
 [3759 3054 2057 ...    0    0    0]
 [3690 1791 2426 ...    0    0    0]
 [2275 4214  288 ...    0    0    0]]


In [21]:
print(embedded_docs[1],'\n', embedded_docs[0])

[3668 3330  288 4902 3549 4747 2534    0    0    0    0    0    0    0
    0    0    0    0    0    0] 
 [1235  827 2514 2524  820 2474 4636 4512  929 4746    0    0    0    0
    0    0    0    0    0    0]


## Creating model

In [22]:
embedding_vector_features=40 ##features representation
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(LSTM(100))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 20, 40)            200000    
                                                                 
 lstm (LSTM)                 (None, 100)               56400     
                                                                 
 dense (Dense)               (None, 1)                 101       
                                                                 
Total params: 256,501
Trainable params: 256,501
Non-trainable params: 0
_________________________________________________________________
None


In [23]:
len(embedded_docs),y.shape

(18285, (18285,))

In [24]:
X_final=np.array(embedded_docs)
y_final=np.array(y)

In [25]:
X_final.shape,y_final.shape

((18285, 20), (18285,))

## Train Test Split

In [26]:
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=42)

## Model Training

In [27]:
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

Epoch 1/10


192/192 [==============================] - 6s 21ms/step - loss: 0.3096 - accuracy: 0.8543 - val_loss: 0.2194 - val_accuracy: 0.9165
Epoch 2/10
192/192 [==============================] - 4s 19ms/step - loss: 0.1291 - accuracy: 0.9527 - val_loss: 0.1977 - val_accuracy: 0.9200
Epoch 3/10
192/192 [==============================] - 4s 20ms/step - loss: 0.0842 - accuracy: 0.9716 - val_loss: 0.2211 - val_accuracy: 0.9118
Epoch 4/10
192/192 [==============================] - 4s 21ms/step - loss: 0.0535 - accuracy: 0.9833 - val_loss: 0.3418 - val_accuracy: 0.9127
Epoch 5/10
192/192 [==============================] - 4s 19ms/step - loss: 0.0398 - accuracy: 0.9880 - val_loss: 0.3100 - val_accuracy: 0.9079
Epoch 6/10
192/192 [==============================] - 4s 19ms/step - loss: 0.0279 - accuracy: 0.9913 - val_loss: 0.3137 - val_accuracy: 0.9127
Epoch 7/10
192/192 [==============================] - 4s 21ms/step - loss: 0.0202 - accuracy: 0.9945 - val_loss: 0.3746 - val_accuracy: 0.9112
Epoch 8/10

## Performance Metrics And Accuracy

In [28]:
y_pred=model.predict(X_test)

189/189 [==============================] - 1s 5ms/step


In [29]:
y_pred=np.where(y_pred > 0.6, 1,0) ##AUC ROC Curve

In [30]:
confusion_matrix(y_test,y_pred)

array([[3143,  276],
       [ 316, 2300]], dtype=int64)

In [31]:
accuracy_score(y_test,y_pred)

0.9019055509527755

In [32]:
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.91      0.92      0.91      3419
           1       0.89      0.88      0.89      2616

    accuracy                           0.90      6035
   macro avg       0.90      0.90      0.90      6035
weighted avg       0.90      0.90      0.90      6035



### Adding Dropout

In [33]:
# from tensorflow.keras.layers import Dropout
# ## Creating model
# embedding_vector_features=40
# model=Sequential()
# model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
# model.add(Dropout(0.3))
# model.add(LSTM(100))
# model.add(Dropout(0.3))
# model.add(Dense(1,activation='sigmoid'))
# model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])